In [ ]:
from rdflib import Namespace

from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from buildingmotif.namespaces import bind_prefixes

In [ ]:
# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://")

# create the model w/ a namespace
BLDG = Namespace("urn:ex/")
bldg = Model.create(BLDG)
bind_prefixes(bldg.graph)
bldg.graph.bind("bldg", BLDG)

In [ ]:
# load libraries
# NREL templates for 223P
nrel_lib = Library.load(directory="../libraries/ashrae/223p/nrel-templates")
# 223P ontology
s223 = Library.load(ontology_graph="../libraries/ashrae/223p/ontology/223p.ttl")

In [ ]:
# get some templates from the library
mau_templ = nrel_lib.get_template_by_name("makeup-air-unit").inline_dependencies()
vav_templ = nrel_lib.get_template_by_name("vav-reheat").inline_dependencies()
duct_templ = nrel_lib.get_template_by_name("duct").inline_dependencies()

In [ ]:
# create a makeup air unit
mau = mau_templ.evaluate({"name": BLDG.MAU, "air-supply": BLDG.MAU_Supply})

In [ ]:
# make 2 VAVs, connect them to the MAU via ducts
vav1 = vav_templ.evaluate({"name": BLDG["VAV-1"], "air-in": BLDG["VAV-1-in"]})
mau_to_vav1 = duct_templ.evaluate({"a": BLDG.MAU_Supply, "b": BLDG["VAV-1-in"]})

vav2 = vav_templ.evaluate({"name": BLDG["VAV-2"], "air-in": BLDG["VAV-2-in"]})
mau_to_vav2 = duct_templ.evaluate({"a": BLDG.MAU_Supply, "b": BLDG["VAV-2-in"]})

In [ ]:
# fill in all the extra parameters with invented names
fill_in = [mau, vav1, vav2, mau_to_vav1, mau_to_vav2]
for templ in fill_in:
    _, graph = templ.fill(BLDG)
    bldg.add_graph(graph)

In [ ]:
# look at finished model
print(bldg.graph.serialize())

In [ ]:
# validate against 223P
ctx = bldg.validate([s223.get_shape_collection()])
print(ctx.valid)
print(ctx.report_string)